In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sns

import df_visualizations as dv
import remap_values as rv

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

In [2]:
%matplotlib inline
plt.style.use( 'dark_background' )
plt.rcParams['figure.figsize'] = (14,10)

In [3]:
use_list = [
    'logerror','transactiondate','airconditioningtypeid','pooltypeid10', # Hotub
    'poolsizesum','garagetotalsqft','calculatedbathnbr','fireplacecnt',
    'finishedsquarefeet12','lotsizesquarefeet','taxdelinquencyflag','yearbuilt',
    'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
    'decktypeid','numberofstories','buildingqualitytypeid','unitcnt','roomcnt' ]

In [4]:
prop  = pd.read_csv( 'data/properties_2016.csv' )
trans = pd.read_csv( 'data/train_2016_v2.csv'   )

my_df = pd.merge( prop, trans, on='parcelid', how='inner' )
my_df = my_df[ use_list ]

del prop
del trans

my_df['transactiondate' ] =   pd.to_datetime  ( my_df['transactiondate'] )
my_df['time_since_built'] = ( pd.DatetimeIndex( my_df['transactiondate'] ).year - my_df['yearbuilt'] )
my_df['ordinal_date'    ] =  [x.toordinal() for x in my_df['transactiondate']]
my_df['day_of_week'     ] = ( my_df['ordinal_date'] - my_df['ordinal_date'].min() + 2 ) % 7

my_df = my_df.drop( ['yearbuilt'], axis=1 )

print my_df.shape

/home/sean/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(90275, 23)


In [5]:
my_df.columns.values

array(['logerror', 'transactiondate', 'airconditioningtypeid',
       'pooltypeid10', 'poolsizesum', 'garagetotalsqft',
       'calculatedbathnbr', 'fireplacecnt', 'finishedsquarefeet12',
       'lotsizesquarefeet', 'taxdelinquencyflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'decktypeid',
       'numberofstories', 'buildingqualitytypeid', 'unitcnt', 'roomcnt',
       'time_since_built', 'ordinal_date', 'day_of_week'], dtype=object)

In [6]:
# Only flag which units have AC

my_df['has_ac'] = 1

my_df.ix[ my_df['airconditioningtypeid'].isnull(), ['has_ac'] ] = 0
my_df.ix[ my_df['airconditioningtypeid'] == 5    , ['has_ac'] ] = 0

my_df = my_df.drop( 'airconditioningtypeid', axis=1 )




# Flag for has pool, has spa, size of pool

my_df['has_spa'  ] = 0
my_df['has_pool' ] = 0
my_df['pool_sqft'] = 0

my_df['has_spa'  ] = my_df['pooltypeid10'].fillna(0).astype(int)
my_df['pool_sqft'] = my_df['poolsizesum' ]
my_df['has_pool' ] =(my_df['poolsizesum' ] > 0).astype(int)

my_df = my_df.drop( ['pooltypeid10','poolsizesum'], axis=1 )




# Save the sqft in better variables

my_df['unit_sqft'] = my_df['finishedsquarefeet12']
my_df[ 'lot_sqft'] = my_df['lotsizesquarefeet'   ]

my_df = my_df.drop( ['finishedsquarefeet12', 'lotsizesquarefeet'], axis=1 )





# Flag for tax delinquency

my_df['tax_delinquent'] = my_df['taxdelinquencyflag'].fillna(0).replace( {'Y', 1} )

my_df = my_df.drop( 'taxdelinquencyflag', axis=1 )





# Break building quality into categories

my_df['building_quality_low' ] = 0
my_df['building_quality_med' ] = 0
my_df['building_quality_high'] = 0
my_df['building_quality_unkn'] = 0

my_df.ix[  my_df['buildingqualitytypeid'] < 4     , 'building_quality_high' ] = 1
my_df.ix[  my_df['buildingqualitytypeid'] > 8     , 'building_quality_low'  ] = 1
my_df.ix[ (my_df['buildingqualitytypeid'] > 3)    &
        (  my_df['buildingqualitytypeid'] < 9)    , 'building_quality_med'  ] = 1
my_df.ix[  my_df['buildingqualitytypeid'].isnull(), 'building_quality_unkn' ] = 1

my_df = my_df.drop( 'buildingqualitytypeid', axis=1 )





# Have flag for garage, and sqft variable

my_df['has_garage' ] = 0
my_df['garage_sqft'] = my_df['garagetotalsqft'].fillna(0)

my_df.ix[ my_df['garagetotalsqft']>1, 'has_garage' ] = 1

my_df = my_df.drop( 'garagetotalsqft', axis=1 )





# 1 hot encode deck, var for number of fireplaces

my_df['n_fireplaces'] = my_df['fireplacecnt'].fillna(0)
my_df['has_deck'    ] = my_df['decktypeid'  ].notnull().astype(int)

my_df = my_df.drop( ['fireplacecnt','decktypeid'], axis=1 )

In [7]:
my_df.columns.values

array(['logerror', 'transactiondate', 'calculatedbathnbr',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'numberofstories', 'unitcnt',
       'roomcnt', 'time_since_built', 'ordinal_date', 'day_of_week',
       'has_ac', 'has_spa', 'has_pool', 'pool_sqft', 'unit_sqft',
       'lot_sqft', 'tax_delinquent', 'building_quality_low',
       'building_quality_med', 'building_quality_high',
       'building_quality_unkn', 'has_garage', 'garage_sqft',
       'n_fireplaces', 'has_deck'], dtype=object)